In [1]:
import cv2
import numpy as np
import os
from skimage.restoration import denoise_wavelet

In [2]:
def preprocess_image(image_path, target_size=(1024, 1024)):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    resized_image    = cv2.resize(image, target_size)
    denoised_image = denoise_wavelet(resized_image, method='BayesShrink', mode='soft')
    return denoised_image

In [3]:
def extract_prnu(image):
    denoised_image = denoise_wavelet(image, method='BayesShrink', mode='soft')
    prnu = image.astype(np.float32) - denoised_image
    return prnu

In [4]:
def average_prnu(camera_images, target_size=(1024, 1024)):
    prnu_sum = np.zeros_like(preprocess_image(camera_images[0], target_size=target_size))  
    for image_path in camera_images:
        prnu_sum += extract_prnu(preprocess_image(image_path, target_size=target_size))
    avg_prnu = prnu_sum / len(camera_images)
    return avg_prnu

In [5]:
def build_prnu_reference(camera_image_dict, target_size=(1024, 1024)):
    prnu_reference = {}
    for camera_id, images in camera_image_dict.items():
        prnu_reference[camera_id] = average_prnu(images, target_size=target_size)
    return prnu_reference

In [6]:
def compute_ncc(prnu1, prnu2):
    prnu1_mean = np.mean(prnu1)
    prnu2_mean = np.mean(prnu2)
    
    prnu1_normalized = prnu1 - prnu1_mean
    prnu2_normalized = prnu2 - prnu2_mean
    
    ncc_value = np.sum(prnu1_normalized * prnu2_normalized) / (np.sqrt(np.sum(prnu1_normalized ** 2) * np.sum(prnu2_normalized ** 2)))
    return ncc_value

In [7]:
def identify_camera(unknown_image_path, prnu_reference, target_size=(1024, 1024)):
    unknown_image = preprocess_image(unknown_image_path, target_size=target_size)
    unknown_prnu = extract_prnu(unknown_image)
    
    ncc_results = {}  
    
    for camera_id, ref_prnu in prnu_reference.items():
        ncc_value = compute_ncc(unknown_prnu, ref_prnu)
        ncc_results[camera_id] = ncc_value
    
    for camera_id, ncc_value in ncc_results.items():
        print(f'Camera: {camera_id} - NCC: {ncc_value}')
    
    best_camera = max(ncc_results, key=ncc_results.get)
    best_ncc = ncc_results[best_camera]
    
    return best_camera, best_ncc

In [8]:
def get_image_paths(folder):
    image_paths = []
    for file_name in os.listdir(folder):
        if file_name.lower().endswith(('.JPG', '.jpg')):
            image_paths.append(os.path.join(folder, file_name))
    return image_paths

In [9]:
camera_folders = {
    'HTC-1-M7': 'C:/Users/PRANAY LANKA/train2/HTC-1-M7',
    'iPhone-4s': 'C:/Users/PRANAY LANKA/train2/iPhone-4s',
    'iPhone-6': 'C:/Users/PRANAY LANKA/train2/iPhone-6',
    'LG-Nexus-5x': 'C:/Users/PRANAY LANKA/train2/LG-Nexus-5x',
    'Motorola-Droid-Maxx': 'C:/Users/PRANAY LANKA/train2/Motorola-Droid-Maxx',
    'Motorola-Nexus-6': 'C:/Users/PRANAY LANKA/train2/Motorola-Nexus-6',
    'Samsung-Galaxy-Note3': 'C:/Users/PRANAY LANKA/train2/Samsung-Galaxy-Note3',
    'Samsung-Galaxy-S4': 'C:/Users/PRANAY LANKA/train2/Samsung-Galaxy-S4',
    'Sony-NEX-7': 'C:/Users/PRANAY LANKA/train2/Sony-NEX-7',
    'Motorola-X': 'C:/Users/PRANAY LANKA/train2/Motorola-X',
    'canon': 'C:/Users/PRANAY LANKA/train2/canon',
}

In [10]:
camera_image_dict = {camera_id: get_image_paths(folder) for camera_id, folder in camera_folders.items()}

prnu_reference = build_prnu_reference(camera_image_dict, target_size=(1024, 1024))

unknown_image_path = 'C:/Users/PRANAY LANKA/train2/IMG_3157.JPG'
identified_camera, ncc_value = identify_camera(unknown_image_path, prnu_reference, target_size=(1024, 1024))

print(f'Identified Camera: {identified_camera} with normalized cross-correlation: {ncc_value}')

Camera: HTC-1-M7 - NCC: -0.00033819592816669396
Camera: iPhone-4s - NCC: 4.551891927832095e-05
Camera: iPhone-6 - NCC: -0.0009595784815474466
Camera: LG-Nexus-5x - NCC: 8.713530918833169e-05
Camera: Motorola-Droid-Maxx - NCC: 0.00045122981985982216
Camera: Motorola-Nexus-6 - NCC: 0.0018287993950376552
Camera: Samsung-Galaxy-Note3 - NCC: 0.0010014293264812941
Camera: Samsung-Galaxy-S4 - NCC: -0.0004136110415223915
Camera: Sony-NEX-7 - NCC: -0.002502786122756878
Camera: Motorola-X - NCC: -0.0022305974268464666
Camera: canon - NCC: 0.014241736073279344
Identified Camera: canon with normalized cross-correlation: 0.014241736073279344
